<a href="https://colab.research.google.com/github/faezesarlakifar/test/blob/main/AAC_feature_vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## We utilized the [source code of the ProAll-D research study](https://data.mendeley.com/datasets/tjmt97xpjf/1) to define descriptors and extract AAC feature vectors. ❤

In [ ]:
# @markdown config
!pip install Bio

In [ ]:
# @title Import Necessaries
import pandas as pd
from Bio import SeqIO
import numpy as np

## Define Descriptors

In [ ]:
E_descriptors = {'A': [0.008,0.134,-0.475,-0.039,0.181],
                 'R': [0.171,-0.361,0.107,-0.258,-0.364],
                 'N': [0.255,0.038,0.117,0.118,-0.055],
                 'D': [0.303,-0.057,-0.014,0.225,0.156],
                 'C': [-0.132,0.174,0.07,0.565,-0.374],
                 'Q': [0.149,-0.184,0.03,0.035,-0.112],
                 'E': [0.221,-0.280,-0.315,0.157,0.303],
                 'G': [0.218,0.562,-0.024,0.018,0.106],
                 'H': [0.023,-0.177,0.041,0.28,-0.021],
                 'I': [-0.353,0.071,-0.088,-0.195,-0.107],
                 'L': [-0.267,0.018,-0.265,-0.274,0.206],
                 'K': [0.243,-0.339,-0.044,-0.325,-0.027],
                 'M': [-0.239,-0.141,-0.155,0.321,0.077],
                 'F': [-0.329,-0.023,0.072,-0.002,0.208],
                 'P': [0.173,0.286,0.407,-0.215,0.384],
                 'S': [0.199,0.238,-0.015,-0.068,-0.196],
                 'T': [0.068,0.147,-0.015,-0.132,-0.274],
                 'W': [-0.296,-0.186,0.389,0.083,0.297],
                 'Y': [-0.141,-0.057,0.425,-0.096,-0.091],
                 'V': [-0.274,0.136,-0.187,-0.196,-0.299]}

In [ ]:
len(E_descriptors)

20

In [ ]:
allergen_label = 1
non_allergen_label = 0

# j_list is 0 to 4 because we have 5 E Descriptors
j_list = [0,1,2,3,4]

# lag is set to 5, as the in our data the length of shortest sequence is 5
lag = [1,2,3,4,5]

# we need this, exclusively for calculating cross covariance
k_list = [0,1,2,3,4]

In [ ]:
# replacing bases in sequences with E descriptors
def represent_as_E_descriptors(allergens):
    allergens_E_descriptors = {}
    for key in allergens:
        allergens_E_descriptors[key] = []

    for key in allergens_E_descriptors:
        for base in allergens[key]:
            allergens_E_descriptors[key].extend(E_descriptors[base])

    return allergens_E_descriptors

In [ ]:
def represent_as_E_descriptors(sequence):

  E = []

  for base in sequence:
    E.extend(E_descriptors[base])

  return E

In [ ]:
# converting E descriptors as a 2-D array, one row corresponding each type of descriptor, for each sequence
def convert_to_2D(descriptors):
    E1 = []
    E2 = []
    E3 = []
    E4 = []
    E5 = []
    for i in range(0,len(descriptors),5):
        E1.append(descriptors[i])
        E2.append(descriptors[i+1])
        E3.append(descriptors[i+2])
        E4.append(descriptors[i+3])
        E5.append(descriptors[i+4])

    E = [E1,E2,E3,E4,E5]
    E = np.array(E)
    return E

## Calculate auto-covariance and cross-covariance

In [ ]:
def calculate_auto_covariance(j_list, lag, n, E):
    auto_covariance = {}
    for j in j_list:
        for l in lag:
            sum = 0
            for i in range(0,n-l):
                sum += (E[j,i] * E[j,i+l])/(n-l)
            key = 'AC'+str(j+1)+str(j+1)+str(l)
            auto_covariance[key] = sum
    #print(auto_covariance)
    return auto_covariance

In [ ]:
def calculate_cross_covariance(j_list, k_list, lag, n, E):
    cross_covariance = {}
    for j in j_list:
        for k in k_list:
            if j == k:
                continue
            else:
                for l in lag:
                    sum = 0
                    for i in range(0,n-l):
                        sum += (E[j,i] * E[k,i+l])/(n-l)
                    key = 'AC'+str(j+1)+str(k+1)+str(l)
                    cross_covariance[key] = sum
    #print(cross_covariance)
    return cross_covariance

In [ ]:
# @title Load fasta files
sequences_valid_pos = SeqIO.parse("algpred2_validation_positive.fasta", "fasta")
sequences_valid_neg = SeqIO.parse("algpred2_validation_negative.fasta", "fasta")
sequences_train_pos = SeqIO.parse("algpred2_train_positive.fasta", "fasta")
sequences_train_neg = SeqIO.parse("algpred2_train_negative.fasta", "fasta")

## Extract AAC features

In [ ]:
def AAC_feature_extractor(sequences, label):
    # Extract features
    feature_rows = []

    for record in sequences:

      sequence = str(record.seq)
      ID = record.id

      E = represent_as_E_descriptors(sequence)
      E_2D = convert_to_2D(E)

      n = len(E)//5
      auto_cov = calculate_auto_covariance(j_list, lag, n, E_2D)
      cross_cov = calculate_cross_covariance(j_list, k_list, lag, n, E_2D)

      row = {**auto_cov, **cross_cov, "Label": label, "id": ID}

      feature_rows.append(row)

    # Create dataframe
    df = pd.DataFrame(feature_rows)

    # Write CSV
    df.to_csv("output.csv", index=False)
    return df

In [ ]:
df_valid_pos = AAC_feature_extractor(sequences_valid_pos, 1)
df_valid_neg = AAC_feature_extractor(sequences_valid_neg, 0)
df_train_pos = AAC_feature_extractor(sequences_train_pos, 1)
df_train_neg = AAC_feature_extractor(sequences_train_neg, 0)

## Merge and Save final feature vectors

In [ ]:
df_valid_pos.head()

,AC111,AC112,AC113,AC114,AC115,AC221,AC222,AC223,AC224,AC225,...,AC533,AC534,AC535,AC541,AC542,AC543,AC544,AC545,Label,id
0,0.001886,0.001307,0.000734,-0.002880,0.000974,-0.001117,-0.000485,0.001613,-0.001169,0.002591,...,-0.000073,0.000695,-0.003275,-0.004461,0.001704,0.000248,0.002862,-0.004252,1,P_7275
1,0.002596,0.000869,-0.000484,-0.003076,0.000697,-0.001412,-0.001544,0.001067,-0.001771,0.002240,...,-0.000084,-0.000580,-0.002765,-0.004671,0.001425,-0.000495,0.003653,-0.004523,1,P_7276
2,0.004823,0.003761,0.003679,0.000331,0.000563,0.009573,0.004162,0.002779,0.005414,0.002088,...,0.000390,-0.004398,-0.001783,-0.004326,-0.002584,-0.001865,-0.002965,-0.002761,1,P_9753
3,0.003292,-0.001578,0.002206,0.000291,0.003010,0.001688,-0.001249,-0.005438,-0.008225,-0.007999,...,0.000138,0.002905,0.002810,-0.003673,-0.000583,0.000653,0.003675,0.003971,1,P_269
4,0.003209,-0.000163,0.001873,-0.001637,0.003451,0.000507,-0.001969,-0.005502,-0.007855,-0.007506,...,0.001537,0.003198,0.001952,-0.003549,-0.001662,-0.000398,0.002570,0.003676,1,P_270


In [ ]:
df_valid_neg.head()

,AC111,AC112,AC113,AC114,AC115,AC221,AC222,AC223,AC224,AC225,...,AC533,AC534,AC535,AC541,AC542,AC543,AC544,AC545,Label,id
0,0.000186,-0.000191,0.000895,0.002556,-0.000403,0.001236,0.002483,0.003533,0.000148,0.000403,...,-0.000199,-0.002812,-0.001541,-0.001318,-0.002495,-0.001670,-0.001676,-0.004209,0,N_147348
1,0.003745,0.006054,0.004899,0.006720,0.005032,-0.001365,0.000260,-0.002811,0.001784,-0.002621,...,0.001081,-0.000429,-0.002312,-0.000987,-0.003047,-0.002983,0.002020,0.000422,0,N_454826
2,0.006231,0.003685,0.006608,0.006939,0.003524,0.005443,0.005607,0.001187,0.005378,0.005761,...,-0.000743,-0.001092,-0.001847,0.000414,-0.001013,0.000444,-0.002130,-0.000500,0,N_158963
3,0.002411,-0.003139,0.000892,0.002348,-0.001100,0.004154,0.000904,-0.001527,0.001662,0.001271,...,0.000069,0.000147,0.001422,0.001029,-0.002634,0.000219,0.003399,0.003215,0,N_65579
4,-0.001021,-0.006761,0.001131,-0.007444,0.004468,0.008914,0.010791,0.011733,0.015616,0.009831,...,-0.002291,-0.003238,0.010797,0.000712,-0.005192,-0.004026,0.003001,0.004504,0,N_262032


In [ ]:
df_train_pos.head()

,AC111,AC112,AC113,AC114,AC115,AC221,AC222,AC223,AC224,AC225,...,AC533,AC534,AC535,AC541,AC542,AC543,AC544,AC545,Label,id
0,0.009383,0.008322,0.007088,0.005579,0.006895,-0.001336,0.002604,0.004305,0.003685,0.003290,...,-0.001084,0.002212,0.001524,0.005349,0.001335,0.001866,0.001508,-0.002260,1,P_13
1,0.000490,-0.001669,0.004654,0.002188,0.000739,0.003232,0.014459,0.008256,0.003990,0.001906,...,-0.000709,0.005907,-0.002400,0.000959,-0.005614,-0.001501,-0.003449,-0.005833,1,P_14
2,0.002151,-0.007809,0.004057,0.000639,-0.004297,-0.001438,0.006269,-0.005696,0.001486,0.007832,...,-0.006384,0.008787,-0.004440,0.000627,0.001405,0.001562,-0.002815,0.002598,1,P_17
3,-0.003347,-0.006373,-0.000094,0.001515,0.002789,0.008365,0.008908,0.006981,0.007383,0.004335,...,-0.005407,0.003977,-0.009576,-0.000270,-0.002400,0.000502,-0.005031,-0.000897,1,P_46
4,-0.002693,-0.007448,-0.000248,0.000191,0.002411,0.004641,0.008922,0.007195,0.004451,0.001105,...,-0.007051,0.004516,-0.009173,-0.001007,-0.003475,-0.000834,-0.006173,-0.000575,1,P_47


In [ ]:
df_train_neg.head()

,AC111,AC112,AC113,AC114,AC115,AC221,AC222,AC223,AC224,AC225,...,AC533,AC534,AC535,AC541,AC542,AC543,AC544,AC545,Label,id
0,-0.014873,-0.010485,0.009514,-0.010916,0.010085,0.009439,0.007081,0.014015,0.016387,0.006417,...,0.008354,-0.001640,-0.011870,0.002486,-0.007811,0.005816,0.009159,0.003974,0,N_15653
1,0.002802,0.000336,0.002092,0.004080,0.002587,-0.001384,0.019427,-0.008919,0.004669,-0.007166,...,0.004029,0.005986,0.003112,0.007879,0.000025,-0.006039,-0.002816,0.011744,0,N_153477
2,-0.001318,-0.014087,0.005753,0.015468,-0.005976,0.007195,0.006989,0.001822,-0.001131,0.011145,...,0.018016,-0.000941,-0.002570,-0.008602,-0.004531,0.001568,0.000676,0.000378,0,N_52129
3,0.017616,0.000081,-0.001344,0.000028,-0.002588,-0.004809,0.004280,-0.006289,-0.003343,-0.007770,...,0.003017,0.008521,0.008130,0.005102,-0.000075,-0.009144,-0.011554,0.002463,0,N_284262
4,0.003154,0.012648,0.013564,0.003795,0.014369,-0.003671,0.019171,-0.011904,0.001491,-0.001205,...,0.000375,0.000262,0.010304,0.002175,-0.011979,0.003908,-0.000656,-0.006815,0,N_458959


In [ ]:
# @title Merge Validation Dataframes
df_valid_AAC = pd.concat([df_valid_neg, df_valid_pos], ignore_index=True, axis=0)

In [ ]:
df_valid_AAC = df_valid_AAC.sample(frac = 1)
df_valid_AAC.head()

,AC111,AC112,AC113,AC114,AC115,AC221,AC222,AC223,AC224,AC225,...,AC533,AC534,AC535,AC541,AC542,AC543,AC544,AC545,Label,id
3456,0.002624,0.006470,0.002608,0.004967,0.004951,0.019992,0.021952,0.022755,0.021410,0.022945,...,0.002324,0.004584,0.002054,-0.003368,-0.000529,-0.005784,-0.003549,-0.003501,1,P_9442
3203,-0.006661,-0.003126,0.002786,-0.004657,0.004363,-0.007840,0.004575,-0.006647,0.001494,-0.003906,...,0.005176,-0.003489,0.002211,-0.004341,0.002747,-0.005422,0.000788,-0.003550,1,P_6308
3542,0.003352,0.000733,-0.001895,0.000282,0.009902,0.003358,0.004605,0.005894,0.004258,0.006740,...,-0.001510,-0.000469,-0.001925,-0.000277,0.000897,-0.001555,-0.000389,0.001086,1,P_5609
2278,-0.004833,0.005966,-0.003482,0.001975,-0.000652,0.008895,0.009987,0.006889,0.008297,0.005965,...,0.003785,-0.001776,-0.000406,-0.001597,0.002127,-0.008022,-0.002905,-0.001883,1,P_3900
1344,0.003772,0.000644,0.001577,0.002191,-0.002938,0.004852,0.003998,0.011075,0.007395,0.005330,...,0.000919,0.002051,-0.003082,0.004117,0.003029,0.000473,0.003191,0.006430,0,N_456440


In [ ]:
df_valid_AAC.to_csv("df_valid_AAC.csv", index=False)

In [ ]:
# @title Merge Validation Dataframes
df_train_AAC = pd.concat([df_train_neg, df_train_pos], ignore_index=True, axis=0)

In [ ]:
df_train_AAC = df_train_AAC.sample(frac = 1)
df_train_AAC.head()

,AC111,AC112,AC113,AC114,AC115,AC221,AC222,AC223,AC224,AC225,...,AC533,AC534,AC535,AC541,AC542,AC543,AC544,AC545,Label,id
15086,0.001287,-0.003737,0.004184,0.004934,0.002597,0.005124,0.004059,-0.002822,-0.001064,0.009070,...,0.007424,0.007778,0.004185,0.003703,0.011624,0.005927,0.004203,-0.000232,1,P_2716
705,-0.010647,0.001622,-0.000106,-0.005017,0.002413,-0.001609,-0.000589,0.000810,-0.001232,-0.000431,...,-0.003907,0.003002,0.001764,0.001258,0.001528,-0.004643,-0.002053,-0.000387,0,N_36306
6510,-0.000433,-0.018975,0.003073,0.008589,-0.004803,0.011586,-0.005543,0.005068,0.012254,0.001695,...,0.002947,-0.002481,0.001814,0.005288,-0.005832,0.003081,-0.000134,-0.000444,0,N_175149
1524,-0.000518,0.000346,-0.000777,0.007440,-0.000433,-0.002089,-0.000176,-0.002413,-0.002200,0.001625,...,-0.002254,0.000372,0.000034,-0.000289,-0.005155,-0.001492,-0.002195,0.001643,0,N_393732
10256,0.004715,0.008496,0.003084,0.002396,-0.002975,0.002222,0.009298,0.007544,0.002571,0.005287,...,-0.002114,-0.000066,0.002943,-0.000839,-0.002706,-0.004998,-0.003422,-0.003153,1,P_2439


In [ ]:
df_train_AAC.to_csv("df_train_AAC.csv", index=False)